In [1]:
import os
from os import chdir
print(os.getcwd())
chdir('C:\\Users\\eric\\Desktop')

C:\Users\eric


In [2]:
import pandas as pd
import csv
import time
import itertools
import numpy as np


In [3]:
data=pd.read_csv("01.HealthCare_CPC_List.csv")

In [4]:
data.head()

,CPC_list
0,"G06F21/6245, G06F19/323, G06F21/78, G06Q50/24,..."
1,"G06F19/321, G06Q10/10, G06Q50/22"
2,"G05B19/042, G05B2219/23043, G05B2219/23044, G0..."
3,"G06Q50/22, G06F19/322, G06Q10/10, G06Q50/24"
4,"H04M3/5183, H04M3/42323, H04M3/5232, H04M3/54"


In [21]:
len(data)

1120

In [22]:
import csv
import time
import itertools
import numpy as np
# node edge
def MakingEdgeNodelist(keyword_vec,sep_sign):
    # (1) Keyword list (data set for combination)
    keywords_list =[]
    for words in keyword_vec:
        #print("words :", words)
        split_words = words.split(sep_sign) # sep_sign으로 구분된 데이터를 분리하여
        split_words = [ x.replace(" ","") for x in split_words]                                                # 리스트로 저장
        soted_words = sorted(split_words) # ordering
        keywords_list.append(soted_words)

    # (2) Combination
    comb_list = []
    for i in keywords_list:
        for subset in itertools.combinations(i, 2):
            if subset[0] != subset[1]: # 같은 단어의 컴비네이션 제외
                comb_list.append(subset)

    # (3) Make dict & count
    comb_count = {}
    for combi in comb_list:
        comb_count[combi] = comb_count.get(combi,0)+1

    # (4) dic to df
    comb_df = pd.DataFrame()

    ## (4-1) dic_key to list
    comb_keylist = []
    for key in comb_count.keys():
        combkey_tolist = list(key)
        comb_keylist.append(combkey_tolist)

    ## (4-2) dic_key split
    source_list = []
    target_list = []
    for i in range(len(comb_keylist)):
        source = comb_keylist[i][0]
        target = comb_keylist[i][1]

        source_list.append(source)
        target_list.append(target)

    comb_df['Source_Label'] = source_list
    comb_df['Target_Label'] = target_list
    comb_df['weight'] = comb_count.values()
    

    comb_df = comb_df.sort_values('Source_Label',axis=0).reset_index(drop=True)

    # (5) node data
    total_tech_list = []
    for tech_li in keywords_list:
        tech = list(set(tech_li))
        total_tech_list.extend(tech)
    # node_data = pd.concat([comb_df['Source_Label'],comb_df['Target_Label']],axis=0) # 총 출현 기술 ; 잘못됨을 확인함

    node_df = pd.DataFrame()

    node_count = {}
    for node in total_tech_list:
        node_count[node] = node_count.get(node,0)+1

    node_df['label'] = node_count.keys()
    node_df['count'] = node_count.values()
    node_df['id'] = node_df.index
    

    # (6) make id
    node_id = node_df[['label','id']]
    node_id.columns = ['Source_Label','Source']

    merge_sid = pd.merge(comb_df,node_id,how = 'inner')

    node_id.columns = ['Target_Label','Target']

    merge_tid = pd.merge(merge_sid,node_id,how = 'inner') # merge 과정에서 순서가 바뀜

    edge_df = merge_tid.sort_values(['Source_Label','Target_Label'])

    edge_df['Type'] = ['Undirected']*len(edge_df)

    #edge_cols = ['Source','Target','weight','Type','Source_Label','Target_Label']
    #edgelist = edgelist[edge_cols]

    return(edge_df,node_df)


# ipc 개수 2 이상인 데이터 필터링
cpc_len = [ len(x.split(",")) for x in data.CPC_list]

data['cpc_len'] = cpc_len

dataset = data.loc[data.cpc_len>1] # , 로 구분되니 한개짜리는 떨군다.

# edge / node list
edge_list = pd.DataFrame()
node_list = pd.DataFrame()

s_time = time.time()
#타임시리즈와다르게for문이필요가없다.
doc = dataset['CPC_list']

edge_part,node_part = MakingEdgeNodelist(doc,',')

edge_list = pd.concat([edge_list,edge_part],axis=0)

node_list = pd.concat([node_list,node_part],axis=0)
    
e_time = time.time()
print("Execution_time :",e_time - s_time)

#
print(node_list.head())
print(edge_list.head())

('Execution_time :', 0.11400008201599121)
           label  count  id
0      G06F3/023      7   0
1    H04L63/0435      1   1
2       G07C3/00     11   2
3      A61B5/681      7   3
4  A61M2205/3393      2   4
   Source_Label Target_Label  weight  Source  Target        Type
0    A01K11/006   A61B5/1112       1    1905    1140  Undirected
69   A01K11/006    A61B5/117       1    1905     321  Undirected
23   A01K11/006   A61B5/1171       1    1905     140  Undirected
77   A01K11/006    A61B5/411       1    1905    1836  Undirected
27   A01K11/006    A61B5/681       1    1905       3  Undirected


In [23]:
node_list.to_csv("node_list_eric.csv",index=False)
edge_list.to_csv("edge_list_eric.csv",index=False)